In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import MiniBatchKMeans, Birch
from scipy.cluster.hierarchy import dendrogram

import re
import string
import time
from PIL import Image
from collections import Counter

import plotly.express as px

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words as nltk_words
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saxen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saxen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saxen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\saxen\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\saxen\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [80]:
data = pd.read_csv('named_entity_extraction.csv')
data

,combined
0,Ukraine Angry Zelensky Russian Ukrainian
1,Ukraine Jeremy Bowen Irpin Russian
2,Ukraine One
3,Manchester Arena Roussos Manchester Arena
4,Ukraine PM Boris Johnson Canadian Dutch
...,...
29140,Bell England New Zealand Lauren Bell England N...
29141,British Andy Murray Wimbledon Thursday Jamie t...
29142,Mertens Emma Raducanu- Elise Mertens third Wim...
29143,John White the Cup Winners Cup


## Vectorization

In [81]:
cv = CountVectorizer(analyzer='word')
cv_data = cv.fit_transform(data['combined'])
dtm_data = pd.DataFrame(cv_data.toarray(), columns=cv.get_feature_names_out())
dtm_data.index = data.index
dtm_data.head(5)

,aa,aaa,aaliyah,aamir,aanoch,aardman,aaron,aarons,aarwangen,aastha,...,zuu,zuwara,zverev,zwelithini,zwerner,zúñiga,élysée,élysées,úsuga,šefčovič
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
word_freq_data_normalized = dtm_data.T
word_freq_data_normalized

,0,1,2,3,4,5,6,7,8,9,...,29135,29136,29137,29138,29139,29140,29141,29142,29143,29144
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaliyah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aamir,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aanoch,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zúñiga,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
élysée,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
élysées,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
úsuga,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
word_freq_data = pd.DataFrame(word_freq_data_normalized.sum(axis=1).reset_index()).rename(columns={'index': 'word', 0: 'freq'})
word_freq_data

,word,freq
0,aa,1
1,aaa,1
2,aaliyah,2
3,aamir,5
4,aanoch,1
...,...,...
16110,zúñiga,1
16111,élysée,1
16112,élysées,1
16113,úsuga,1


In [84]:
TRIM_QUANTILE = 0.7
word_freq_data[word_freq_data['freq'] >= word_freq_data['freq'].quantile(TRIM_QUANTILE)]

,word,freq
3,aamir,5
5,aardman,4
6,aaron,30
13,abbas,7
14,abbess,4
...,...,...
16100,zuma,3
16103,zurcher,4
16104,zurich,9
16107,zverev,13


In [85]:
stopwords = list(word_freq_data[word_freq_data['freq'] < word_freq_data['freq'].quantile(TRIM_QUANTILE)]['word'])
vec_vocab = list(word_freq_data[word_freq_data['freq'] >= word_freq_data['freq'].quantile(TRIM_QUANTILE)]['word'])

In [86]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words=stopwords, vocabulary=vec_vocab)
tfidf_vectorizer

TfidfVectorizer(stop_words=['aa', 'aaa', 'aaliyah', 'aanoch', 'aarons',
                            'aarwangen', 'aastha', 'ab', 'abattoir', 'abba',
                            'abbie', 'abbkr', 'abbygail', 'abdel', 'abdelmajid',
                            'abdesalem', 'abdollahian', 'abdourahmane',
                            'abdujalil', 'abdulaziz', 'abdulhak', 'abdullah',
                            'abdullatif', 'abdurahman', 'abdurasulov', 'abdy',
                            'abedi', 'abelleira', 'abercombie', 'aberdare', ...],
                vocabulary=['aamir', 'aardman', 'aaron', 'abbas', 'abbess',
                            'abbey', 'abbington', 'abbott', 'abby', 'abdallah',
                            'abdeslam', 'abdul', 'abe', 'abed', 'abercrombie',
                            'aberdeen', 'aberdeenshire', 'aberg', 'aberystwyth',
                            'abigail', 'abortion', 'about', 'above',
                            'abramovich', 'abu', 'abualouf', 'abuja', 'ac',
                            'academic', 'academy', ...])

In [87]:
tfidf_matrix = tfidf_vectorizer.fit_transform(data['combined'])
tfidf_matrix

<29145x5999 sparse matrix of type '<class 'numpy.float64'>'
	with 141539 stored elements in Compressed Sparse Row format>

In [88]:
tfidf_data = pd.DataFrame(tfidf_matrix.toarray())
tfidf_data

,0,1,2,3,4,5,6,7,8,9,...,5989,5990,5991,5992,5993,5994,5995,5996,5997,5998
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
feature_names = tfidf_vectorizer.get_feature_names_out()
feature_names

array(['aamir', 'aardman', 'aaron', ..., 'zurich', 'zverev', 'zwerner'],
      dtype=object)

In [90]:
tfidf_data.columns = feature_names
tfidf_data

,aamir,aardman,aaron,abbas,abbess,abbey,abbington,abbott,abby,abdallah,...,zone,zoo,zoysa,zuckerberg,zulu,zuma,zurcher,zurich,zverev,zwerner
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Word2Vec

In [160]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [161]:
psl_data = pd.read_csv('punctuation_stopwords_lemmatization.csv')
psl_data = psl_data['combined']

In [162]:
tokenized_data = [simple_preprocess(sentence) for sentence in psl_data]
tokenized_data

[['ukraine',
  'angry',
  'zelensky',
  'vow',
  'punish',
  'russian',
  'atrocity',
  'ukrainian',
  'president',
  'say',
  'country',
  'forgive',
  'forget',
  'murder',
  'civilian'],
 ['war',
  'ukraine',
  'take',
  'cover',
  'town',
  'attack',
  'jeremy',
  'bowen',
  'frontline',
  'irpin',
  'resident',
  'come',
  'russian',
  'fire',
  'try',
  'flee'],
 ['ukraine',
  'war',
  'catastrophic',
  'global',
  'food',
  'one',
  'world',
  'biggest',
  'fertiliser',
  'firm',
  'say',
  'conflict',
  'could',
  'deliver',
  'shock',
  'food',
  'supply'],
 ['manchester',
  'arena',
  'bomb',
  'saffie',
  'roussoss',
  'parent',
  'hear',
  'truth',
  'parent',
  'manchester',
  'arena',
  'bombing',
  'youngest',
  'victim',
  'speak',
  'life',
  'since',
  'die'],
 ['ukraine',
  'conflict',
  'oil',
  'price',
  'soar',
  'highest',
  'level',
  'since',
  'consumer',
  'feel',
  'impact',
  'higher',
  'energy',
  'cost',
  'fuel',
  'price',
  'household',
  'bill',
  '

In [163]:
# Train Word2Vec model
model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

In [164]:
# Example: Get the vector for a word
vector = model.wv['ukraine']
print(vector)

[ 1.6243329  -1.7038977   1.0814083   2.316599    0.93031067  0.20072623
  0.78711075 -0.03018015 -1.5763694  -2.2065976  -0.23263837 -0.32052502
  0.45147806  0.5456151   0.99737865  1.102238    0.41510594  0.13483773
 -0.56725127 -0.75768846  0.10246266 -0.88262373  0.7200519  -1.907975
 -0.9198684   0.72295934 -1.5752314   2.7368073  -0.15391123  1.406349
 -0.85208666 -0.0044636  -0.90091974 -1.4320134  -0.01006114 -0.26713338
 -0.69058335  0.89449644 -0.43282795  1.2991616   1.2122707  -0.6026
  0.8251942   1.4070228   1.7013526  -0.31991246 -1.5064955   0.02133384
  1.3725147   0.41696718 -2.0909235  -1.3642356   1.4190829  -2.170979
 -1.411016    2.4829717  -1.8827736   1.4333808   0.7462665   0.628307
  1.018669    0.6426362   0.5610173   3.008826   -0.30733478  1.6023633
 -0.95219266  2.757013   -0.63606167 -0.44578692  1.910418   -0.39981785
  1.7505888   0.16431311 -0.02258592  0.26195663  0.9109732  -0.5266539
 -0.03015067 -2.2636096  -2.3242161   0.60780543 -1.9507964   1.8

In [165]:
model.save("word2vec.model")

In [166]:
model = Word2Vec.load("word2vec.model")

# Information Retrieval

### Using TF-IDF on NER Dataset

In [168]:
from sklearn.metrics.pairwise import cosine_similarity


def retrieve_top_documents(tfidf_data, query_vector, top_n=10):

    cosine_similarities = cosine_similarity(tfidf_data, query_vector)

    cosine_similarities = cosine_similarities.flatten()

    top_document_indices = cosine_similarities.argsort()[-top_n:][::-1]

    top_documents = tfidf_data.iloc[top_document_indices]
    top_scores = cosine_similarities[top_document_indices]
    
    return top_documents, top_scores


In [169]:
query = "Ukrain and Russia war" 
query_vector = tfidf_vectorizer.transform([query])

In [170]:
for i in query_vector.nonzero()[1]:
    print(f'{feature_names[i]}: {query_vector[0, i]}')

war: 0.668486596545688
russia: 0.4463676354192003
and: 0.5948794871980664


In [171]:
top_documents, top_scores = retrieve_top_documents(tfidf_data, query_vector, top_n=10)

for i, (doc, score) in enumerate(zip(top_documents.index, top_scores)):
    print(f"Title : {data.iloc[doc, 0]}, Similarity Score = {score}")


Title : Ukraine Russia The War, Similarity Score = 0.6912881658491632
Title : Srebrenica, a Bosnian War, Similarity Score = 0.668486596545688
Title : Ukraine War Russia Kyiv, Similarity Score = 0.5821152793509781
Title : Ukraine Masyanya Russia Masyanya YouTube Russia the Ukraine war, Similarity Score = 0.530697297835818
Title : Gaza World War years World War One, Similarity Score = 0.5161355078769382
Title : World War Two, Similarity Score = 0.5122937366268991
Title : Ukraine War Russia UN almost, Similarity Score = 0.5121647401270637
Title : Ukraine Putin Russia the Ukraine war, Similarity Score = 0.5083155306928208
Title : World War One, Similarity Score = 0.5059100397546838
Title : Arbacia Two and, Similarity Score = 0.4940053765234577


### Using Word2Vec on PSL Dataset

In [179]:
def get_average_word2vec(sentence, model):

    words = simple_preprocess(sentence)

    valid_words = [word for word in words if word in model.wv.key_to_index]
    
    # If no valid words, return a zero vector
    if not valid_words:
        return np.zeros(model.vector_size)
    
    # Compute the average of the word vectors
    return np.mean([model.wv[word] for word in valid_words], axis=0)

get_average_word2vec("Ukraine and Russia war", model)

array([ 1.48894656e+00, -1.68627322e+00,  2.47861072e-01,  1.76761866e+00,
        1.14339304e+00, -2.92838961e-01,  7.17113674e-01,  7.72134542e-01,
       -1.65365589e+00, -2.00327015e+00, -3.64948601e-01, -3.06785613e-01,
        4.30595011e-01,  1.35704947e+00,  1.25805473e+00,  8.63188028e-01,
        3.45592231e-01,  6.77336156e-01,  3.30763847e-01, -1.48972660e-01,
        5.62618375e-01, -1.59620273e+00,  4.58133221e-01, -1.79121816e+00,
       -1.23363137e-01,  2.42303565e-01, -1.73466504e+00,  2.12482309e+00,
        7.85307467e-01,  7.33723640e-01,  1.80834815e-01,  5.06631285e-02,
       -9.09626424e-01, -9.47862387e-01, -6.56322166e-02, -3.37730259e-01,
       -4.25965339e-01,  9.60309088e-01,  1.59529626e-01,  3.38319659e-01,
        5.79287350e-01, -6.72376871e-01,  1.43225193e+00,  1.50561273e+00,
        7.86463559e-01, -1.54620722e-01, -1.36880398e+00,  2.27708295e-02,
        1.04264212e+00, -6.92228496e-04, -1.32766449e+00, -1.45838070e+00,
        5.66611111e-01, -

In [180]:
document_vectors = np.array([get_average_word2vec(sentence, model) for sentence in psl_data])
document_vectors

array([[ 0.66202915, -0.76915073,  0.28096864, ..., -0.16041747,
         0.08016169, -0.35962346],
       [ 0.61783683, -0.6129756 ,  0.1437464 , ..., -0.20232466,
         0.2055212 , -0.380801  ],
       [ 0.39168555, -0.19382471,  0.06696969, ..., -0.62994206,
         0.15906544, -0.32175714],
       ...,
       [-0.96422017,  0.51544183,  0.48684964, ..., -0.48833603,
         0.45662627, -0.21610163],
       [-0.48947015,  0.25353405,  0.38317508, ..., -0.50422   ,
         0.30455387, -0.13027124],
       [ 0.17837492, -0.10847688,  0.31274453, ..., -0.3395309 ,
         0.11828453, -0.17409433]], dtype=float32)

In [187]:
def retrieve_top_documents_word2vec(document_vectors, query_vector, top_n=10):
    
    cosine_similarities = cosine_similarity(document_vectors, query_vector).flatten()

    top_document_indices = cosine_similarities.argsort()[-top_n:][::-1]

    top_document_indices = cosine_similarities.argsort()[-top_n:][::-1]

    top_documents = psl_data.iloc[top_document_indices]
    top_scores = cosine_similarities[top_document_indices]
    
    return top_documents, top_scores

In [192]:
query = "India in World Cup"
query_vector = get_average_word2vec(query, model).reshape(1, -1)

In [193]:
top_documents, top_scores = retrieve_top_documents_word2vec(document_vectors, query_vector, top_n=10)

for i, (doc, score) in enumerate(zip(top_documents.index, top_scores)):
    print(f"Title : {psl_data.iloc[doc]}, Similarity Score = {score}")

Title : icc woman t20 world cup 2023 sri lanka shock host south africa world cup opener sri lankas bowler produce inspire display shock host south africa opening fixture womens t20 world cup, Similarity Score = 0.985902726650238
Title : india crush england reach t20 world cup final england defence t20 world cup slump 68run thrash superb india semifinal guyana, Similarity Score = 0.9855734705924988
Title : cricket world cup 2023 india pakistan ready match 132000seater stadium india pakistan set epic encounter cricket world cup saturday host world largest cricket stadium, Similarity Score = 0.9854472875595093
Title : woman world cup england reach final thanks wyatts 129 ecclestones six wicket danni wyatts magnificent century sophie ecclestones 636 propels england 137run defeat south africa womens world cup final, Similarity Score = 0.9853226542472839
Title : world cup 2022 test knowledge qatar 2022 world cup nearly reach conclusion much remember qatar 2022, Similarity Score = 0.985204815